In [64]:
import pandas as pd
import numpy as np
import boto3
import os
from sagemaker import get_execution_role

bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'payment/output'
file_name = 'test_data.csv'

data_location = 's3://{}/{}/{}'.format(bucket_name, prefix_name, file_name)

role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [65]:
test_data_file = f's3://{bucket_name}/{prefix_name}/test/test_data.csv'
print(test_data_file)

s3://eliezerraj-908671954593-dataset/payment/output/test/test_data.csv


In [66]:
df_test_data = pd.read_csv(test_data_file,header=None)
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
df_test_data.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,71,0,1,1,210.0,4,425.00,6,492.00,13,388.31,1022
1,1,234,0,1,1,291.0,4,665.25,4,665.25,5,558.80,98
2,0,101,0,1,1,176.0,1,176.00,3,185.33,8,168.00,0


In [67]:
x_test = df_test_data.iloc[:, 1:]
y_test = df_test_data.iloc[:, :1]

In [68]:
# Extract feature
x_test.head(3)

,1,2,3,4,5,6,7,8,9,10,11,12
0,71,0,1,1,210.0,4,425.00,6,492.00,13,388.31,1022
1,234,0,1,1,291.0,4,665.25,4,665.25,5,558.80,98
2,101,0,1,1,176.0,1,176.00,3,185.33,8,168.00,0


In [69]:
# extract label
y_test.head(3)

,0
0,0
1,1
2,0


In [70]:
from sagemaker import Session

client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")
# Endpoint invocation
endpoint_name="xgboost-serverless-ep-fraud-model-v3-2024-04-14-02-35-35"

payload = b"100., 0., 1., 1., 400., 365.0, 17., 263.529412, 28., 238.714286, 97582.0"

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType="text/csv",
)


In [71]:
response["Body"].read().decode()

'0.14927132427692413'

In [72]:
def predict(data, predictor, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

In [73]:
predictions = predict(x_test, response)

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


AttributeError: 'dict' object has no attribute 'predict'